In [2]:
import pandas as pd

# ----------------------------------------------------------------------
# 1)  Ajuste aqui os caminhos/nomes dos seus dois arquivos CSV
# ----------------------------------------------------------------------
PATH_TREINO = "./data/demo_prepro.csv"   # seu dataset convertido
PATH_CIC    = "./data/cic-output.csv"               # saída do cicflowmeter-python

# ----------------------------------------------------------------------
# 2)  Carrega ambos em DataFrames
# ----------------------------------------------------------------------
df_train = pd.read_csv(PATH_TREINO)
df_cic   = pd.read_csv(PATH_CIC)

# ----------------------------------------------------------------------
# 3)  Interseção dos cabeçalhos
# ----------------------------------------------------------------------
common_cols = df_train.columns.intersection(df_cic.columns)

print(f"\n==> Colunas em comum: {len(common_cols)}\n")
for col in common_cols:
    print(col)

# (opcional) se quiser ver quem ficou de fora em cada um:
missing_in_train = df_cic.columns.difference(df_train.columns)
missing_in_cic   = df_train.columns.difference(df_cic.columns)

print(f"\n==> Só no CSV do CIC-Python ({len(missing_in_train)}):")
print(list(missing_in_train))

print(f"\n==> Só no dataset de treino ({len(missing_in_cic)}):")
print(list(missing_in_cic))


==> Colunas em comum: 69

dst_port
flow_duration
tot_fwd_pkts
tot_bwd_pkts
totlen_fwd_pkts
totlen_bwd_pkts
fwd_pkt_len_max
fwd_pkt_len_min
fwd_pkt_len_mean
fwd_pkt_len_std
bwd_pkt_len_max
bwd_pkt_len_min
bwd_pkt_len_mean
bwd_pkt_len_std
flow_byts_s
flow_pkts_s
fwd_pkts_s
bwd_pkts_s
pkt_len_min
pkt_len_max
pkt_len_mean
pkt_len_std
pkt_len_var
flow_iat_mean
flow_iat_std
flow_iat_max
flow_iat_min
fwd_iat_tot
fwd_iat_mean
fwd_iat_std
fwd_iat_max
fwd_iat_min
bwd_iat_tot
bwd_iat_mean
bwd_iat_std
bwd_iat_max
bwd_iat_min
fwd_header_len
bwd_header_len
fwd_seg_size_min
fwd_act_data_pkts
fin_flag_cnt
syn_flag_cnt
rst_flag_cnt
psh_flag_cnt
ack_flag_cnt
urg_flag_cnt
fwd_psh_flags
fwd_urg_flags
ece_flag_cnt
cwr_flag_count
down_up_ratio
pkt_size_avg
init_fwd_win_byts
init_bwd_win_byts
active_max
active_min
active_mean
active_std
idle_max
idle_min
idle_mean
idle_std
fwd_seg_size_avg
bwd_seg_size_avg
subflow_fwd_pkts
subflow_bwd_pkts
subflow_fwd_byts
subflow_bwd_byts

==> Só no CSV do CIC-Python (13):

In [4]:
import numpy as np


discrepancies = []

for col in common_cols:
    dt_train = df_train[col].dtype
    dt_cic   = df_cic[col].dtype

    # mediana apenas para numéricos
    med_train = df_train[col].median() if np.issubdtype(dt_train, np.number) else "—"
    med_cic   = df_cic[col].median()   if np.issubdtype(dt_cic,   np.number) else "—"

    flag = ""
    if np.issubdtype(dt_train, np.number) and np.issubdtype(dt_cic, np.number):
        if med_cic != 0:
            ratio = med_train / med_cic
            if ratio > 1_000 or ratio < 0.001:
                flag = "  **WARNING: escalas muito diferentes**"
                discrepancies.append(col)
    elif dt_train != dt_cic:
        flag = "  **WARNING: tipos diferentes**"
        discrepancies.append(col)

    print(f"{col:25}  train: {dt_train} | cic: {dt_cic} | "
          f"median(train)={med_train}  median(cic)={med_cic}{flag}")

# -------------------------------------------------
# resumo
# -------------------------------------------------
if discrepancies:
    print("\n>>> Colunas que precisam atenção:", discrepancies)
else:
    print("\n>>> Todos os tipos e grandezas batem!")

dst_port                   train: int64 | cic: int64 | median(train)=66.5  median(cic)=443.0
flow_duration              train: float64 | cic: float64 | median(train)=0.020288  median(cic)=93.6728219985962  **WARNING: escalas muito diferentes**
tot_fwd_pkts               train: int64 | cic: int64 | median(train)=2.0  median(cic)=19.0
tot_bwd_pkts               train: int64 | cic: int64 | median(train)=2.0  median(cic)=16.0
totlen_fwd_pkts            train: int64 | cic: int64 | median(train)=55.0  median(cic)=5189.0
totlen_bwd_pkts            train: int64 | cic: int64 | median(train)=107.0  median(cic)=5071.0
fwd_pkt_len_max            train: int64 | cic: int64 | median(train)=33.0  median(cic)=1972.0
fwd_pkt_len_min            train: int64 | cic: int64 | median(train)=14.5  median(cic)=66.0
fwd_pkt_len_mean           train: float64 | cic: float64 | median(train)=33.0  median(cic)=216.1153846153846
fwd_pkt_len_std            train: float64 | cic: float64 | median(train)=0.0  median(cic)=